# A fractional ﬁlter based efﬁcient algorithm for retinal blood vessel
# segmentation

Fractional filter implementation


In [1]:
import scipy.special as sc
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def lower_gammainc(alpha,a):
    return 1 - sc.gammaincc(alpha, a)
def gamma_func(n):
    return sc.gamma(n)
def print_matrix(matrix):
    for row in matrix:
        for val in row:
            print ('{:4}'.format(val),end=" "),
        print()    
        

    

In [3]:
def get_coefficient(n,k,alpha,a):
    if k==0 :
        return lower_gammainc(alpha,a)/(2*pow(a,alpha)*gamma_func(alpha))
    if k==1 :
        return lower_gammainc(alpha,2*a)/(2*pow(a,alpha)*gamma_func(alpha))
    if k <= n-1 :
        return (lower_gammainc(alpha,(k+1)*a) - lower_gammainc(alpha,(k-1)*a))/(2*pow(a,alpha)*gamma_func(alpha))
    return (lower_gammainc(alpha,(n)*a) - lower_gammainc(alpha,(n-1)*a))/(2*pow(a,alpha)*gamma_func(alpha))
        
    
    

In [4]:
def build_mask(size,alpha,a):
    coefficients=[]
    for i in range(size+1):
        coefficients.append(get_coefficient(size,i,alpha,a))
    
    mask = np.zeros((2*size+1,2*size+1))
    mask[size][size]=coefficients[0]
    dx=[0,1,1,1,0,-1,-1,-1]
    dy=[1,1,0,-1,-1,-1,0,1]
    for i in range(1,size+1):
        for direction in range(8):
            mask[size+i*dx[direction]][size+i*dy[direction]]=coefficients[i]
    return mask        

In [7]:
kern=build_mask(1,0.08,0.85)

In [8]:
print(kern)

[[0.04194259 0.04194259 0.04194259]
 [0.04194259 0.0411975  0.04194259]
 [0.04194259 0.04194259 0.04194259]]


# Principal Component Analysis


In [6]:
def PCA(f,kernal_size):
    #Defining our kernals
    K_XC=np.zeros((kernal_size,kernal_size))
    K_YC=np.zeros((kernal_size,kernal_size))
    for row in range(kernal_size):
        for col in range(kernal_size):
            K_XC[row][col]=col
            K_YC[row][col]=row
    img_size = f.shape[0]
    #Initializing Weighted mean matrix
    X_C=np.zeros((img_size,img_size))
    Y_C=np.zeros((img_size,img_size))
    #Computing weighted mean matrix
    n=kernal_size
    for x in range(img_size):
        for y in range(img_size):
            num_x=0
            num_y=0
            denom=0
            for i in range(kernal_size):
                for j in range(kernal_size):
                    if x+1+i-(n+1)//2 >= img_size or y+1+j-(n+1)//2 >= img_size :
                        continue
                    num_x+=K_XC[i][j]*f[x+1+i-(n+1)//2][y+1+j-(n+1)//2]
                    num_y+=K_YC[i][j]*f[x+1+i-(n+1)//2][y+1+j-(n+1)//2]
                    denom+=f[x+1+i-(n+1)//2][y+1+j-(n+1)//2]
            
            X_C[x][y]=num_x/denom
            Y_C[x][y]=num_y/denom
    
    #Initializing Variance matrix
    X_V=np.zeros((img_size,img_size))
    Y_V=np.zeros((img_size,img_size))
    for x in range(img_size):
        for y in range(img_size):
            for i in range(kernal_size):
                for j in range(kernal_size):
                    if x+1+i-(n+1)//2 >= img_size or y+1+j-(n+1)//2 >= img_size :
                        continue
                    X_V[x][y]+=((K_XC[i][j]-X_C[x][y])**2)*f[x+1+i-(n+1)//2][y+1+j-(n+1)//2]
                    Y_V[x][y]+=((K_YC[i][j]-Y_C[x][y])**2)*f[x+1+i-(n+1)//2][y+1+j-(n+1)//2]
    
    
    #Initializing the covariance matrix
    Cov = np.zeros((img_size,img_size))
    for x in range(img_size):
        for y in range(img_size):
            for i in range(kernal_size):
                for j in range(kernal_size):
                    if x+1+i-(n+1)//2 >= img_size or y+1+j-(n+1)//2 >= img_size :
                        continue
                    Cov[x][y]+=((K_XC[i][j]-X_C[x][y])*(K_YC[i][j]-Y_C[x][y]))*f[x+1+i-(n+1)//2][y+1+j-(n+1)//2]
    
    matrix = [[]for i in range(img_size)]        
    for x in range(img_size):
        for y in range(img_size):
            temp=np.zeros((2,2))
            temp[0][0]=X_V[x][y]
            temp[1][1]=Y_V[x][y]
            temp[0][1]=Cov[x][y]
            temp[1][0]=Cov[x][y]
            matrix[x].append(temp)
    return matrix    
            

In [7]:
size = 75
f = np.zeros((size,size))
for x in range(size):
    for y in range(size):
        f[x][y]=np.random.rand()
matrix=PCA(f,5)      

In [8]:
import cv2

In [9]:
image=cv2.imread("retina.jpeg")
sz=image.shape[0]
kern2=build_mask(sz,0.08,0.85)

In [10]:
print(kern2)

[[[6 4 4]
  [6 4 4]
  [6 4 4]
  ...
  [3 3 3]
  [3 3 3]
  [3 3 3]]

 [[6 4 4]
  [6 4 4]
  [6 4 4]
  ...
  [3 3 3]
  [3 3 3]
  [3 3 3]]

 [[6 4 4]
  [6 4 4]
  [6 4 4]
  ...
  [3 3 3]
  [3 3 3]
  [3 3 3]]

 ...

 [[3 3 3]
  [3 3 3]
  [3 3 3]
  ...
  [3 3 3]
  [3 3 3]
  [3 3 3]]

 [[3 3 3]
  [3 3 3]
  [3 3 3]
  ...
  [3 3 3]
  [3 3 3]
  [3 3 3]]

 [[3 3 3]
  [3 3 3]
  [3 3 3]
  ...
  [3 3 3]
  [3 3 3]
  [3 3 3]]]


In [11]:
dst = cv2.filter2D(image,-1,kern2)

In [12]:
print(dst)

[[[2 2 2]
  [2 2 2]
  [2 2 2]
  ...
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[2 2 2]
  [2 2 2]
  [2 2 2]
  ...
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[2 2 2]
  [2 2 2]
  [2 2 2]
  ...
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 ...

 [[1 1 1]
  [1 1 1]
  [1 1 1]
  ...
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]
  [1 1 1]
  ...
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]
  [1 1 1]
  ...
  [1 1 1]
  [1 1 1]
  [1 1 1]]]


In [13]:
def show(img):
    cv2.imshow('im',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [14]:
show(dst)